In [2]:
import findspark
findspark.init()
#from Ipython.core.interactiveShell import InteractiveShell
#InteractiveShell.ast_node_interactivity = 'all'

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, Row

from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import col, when, explode
import pyspark.sql.functions as F

from pyspark.ml.linalg import Vectors

import pyodbc
#from Ipython.display import display, FileLink, FileLinks

import os
import pandas as pd
from pandas import DataFrame
#pd.option.display.max_columns = None
import numpy as np
import lightgbm as lgb
import xgboost as xgb

In [3]:
spark =SparkSession\
   .builder\
   .appName("test")\
   .enableHiveSupport().getOrCreate()

sc= spark.sparkContext
sqlContext= SQLContext(sc)

findspark.find()

#x = np.genfromtxt("C:/Users/affiqazrin/Desktop/anacondaScripts/data_berka/trans.asc", dtype=None, delimiter=";")

#A = sc.parallelize(x)
#df = pd.DataFrame(data=x)
#df.head(100)

'C:\\spark\\spark-3.0.1-bin-hadoop2.7'

In [4]:
df = sqlContext.read.format("csv")\
   .option("header", "true")\
   .load("C:/Users/affiqazrin/Desktop/spark_tutorials/Data_FinalProject.csv")

In [ ]:
features = pd.read_csv("C:/Users/affiqazrin/Desktop/spark_tutorials/feature_names.csv")
features=features.transpose()
features.column=['fet']

In [5]:
def spark_shape(self):
    return (self.count(), len(self.columns))
pyspark.sql.dataframe.DataFrame.shape = spark_shape

spark_shape(df)
df.printSchema()
df.show()

root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp.var.rate: string (nullable = true)
 |-- cons.price.idx: string (nullable = true)
 |-- cons.conf.idx: string (nullable = true)
 |-- euribor3m: string (nullable = true)
 |-- nr.employed: string (nullable = true)
 |-- y: string (nullable = true)

+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+---------

In [ ]:
from pyspark.ml.feature import OneHotEncoder

df = sqlContext.createDataFrame([
(0, "a"),
(1, "b"),
(2, "c"),
(3, "a"),
(4, "a"),
(5, "c")
], ["id", "category"])

stringIndexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
model = stringIndexer.fit(df)
indexed = model.transform(df)
indexed.show()

encoder = OneHotEncoder(inputCol="categoryIndex", outputCol="vectorIndex")
model = encoder.fit(indexed)
encoded = model.transform(indexed)
encoded.show()

In [ ]:
clss=df.select('y').collect()

In [6]:


categorical_cols = ["job","marital","education","default","housing","loan","contact","month","poutcome"]
categorical_cols2 = ["job","marital"]
numeric_cols = ["age", "day_of_week", "duration", "campaign", "pdays","previous"]

#df.select(categorical_cols).collect()
#df.select(numeric_cols).collect()

In [ ]:
for cols in categorical_cols:
    print(cols)

In [ ]:
from pyspark.ml.feature import OneHotEncoder

for cols in categorical_cols2:
    print(cols)
    stringIndexer = StringIndexer(
        inputCols=[cols + "_index" for cols in categorical_cols2], 
        outputCols=[cols + "_classVec" for cols in categorical_cols2])

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder
stages = []

#iterate through all categorical values
for cols in categorical_cols:
    #create a string indexer for those categorical values and assign a new name including the word 'Index'
    stringIndexer = StringIndexer(inputCol = cols,
                                  outputCol = cols + '_index')

    #append the string Indexer to our list of stages
    stages += [stringIndexer]
    
#Create the pipeline. Assign the satges list to the pipeline key word stages
pipeline = Pipeline(stages = stages)

#fit the pipeline to our dataframe
pipelineModel = pipeline.fit(df)

#transform the dataframe
df= pipelineModel.transform(df)

df.show()

In [ ]:
# Convert Feature Types
df.createOrReplaceTempView("df")

df2 = spark.sql("select \
                    cast(id as int) as id, \
                    cast(age as int) as age, \
                    cast(job as string) as job, \
                    cast(marital as string) as marital, \
                    cast(education as string) as education, \
                    cast(default as string) as default, \
                    cast(balance as int) as balance, \
                    cast(housing as string) as housing, \
                    cast(loan as string) as loan, \
                    cast(contact as string) as contact, \
                    cast(day as int) as day, \
                    cast(month as string) as month, \
                    cast(duration as int) as duration, \
                    cast(campaign as int) as campaign, \
                    cast(pdays as int) as pdays, \
                    cast(previous as int) as previous, \
                    cast(poutcome as string) as poutcome, \
                    cast(deposit as string) as deposit \
                from df")

# Data Types
df2.dtypes

[('id', 'int'),
 ('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('balance', 'int'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'int'),
 ('month', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('deposit', 'string')]


 # Build Pipeline (Error is Here)
categorical_cols = ["job","marital","education","default","housing","loan","contact","month","poutcome"]
numeric_cols = ["age", "balance", "day", "duration", "campaign", "pdays","previous"]

stages = []

stringIndexer = StringIndexer(inputCol=[cols for cols in categorical_cols],
                              outputCol=[cols + "_index" for cols in categorical_cols])

encoder = OneHotEncoderEstimator(inputCols=[cols + "_index" for cols in categorical_cols],
                                 outputCols=[cols + "_classVec" for cols in categorical_cols])

stages += [stringIndexer, encoder]

label_string_id = StringIndexer(inputCol="deposit", outputCol="label")
stages += [label_string_id]

assembler_inputs = [cols + "_classVec" for cols in categorical_cols] + numeric_cols
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")
stages += [assembler]

# Run Data Through Pipeline
pipeline = Pipeline().setStages(stages)
pipeline_model = pipeline.fit(df2)
prepped_df = pipeline_model.transform(df2)

In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer

stages = []

stringIndexer = StringIndexer(inputCol=[cols for cols in categorical_cols],
                              outputCol=[cols + "_index" for cols in categorical_cols])

encoder = OneHotEncoderEstimator(inputCols=[cols + "_index" for cols in categorical_cols],
                                 outputCols=[cols + "_classVec" for cols in categorical_cols])

stages += [stringIndexer, encoder]

label_string_id = StringIndexer(inputCol="deposit", outputCol="label")
stages += [label_string_id]

assembler_inputs = [cols + "_classVec" for cols in categorical_cols] + numeric_cols
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")
stages += [assembler]

# Run Data Through Pipeline
pipeline = Pipeline().setStages(stages)
pipeline_model = pipeline.fit(df2)
prepped_df = pipeline_model.transform(df2)

TypeError: Invalid param value given for param "inputCol". Could not convert <class 'list'> to string type